In [1]:
#---import libraries relating to data manipulation, visualization---------------------------------------------------
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)

import numpy as np
import json

#-------import libraries to evaluate the deep learning model---------------------------------------------------------------
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

#------ import Deep Learning Libraries-------#

import torch
import torch.nn as nn
import torch.optim as optim


import optuna
from optuna.trial import Trial
import pickle


import sys
sys.path.append('../../../src')  # Add the 'src' directory to the path

#### Import Data Loading and Processing Functions<
from multimodal.helper_functions import reproducible_comp, get_final_df
from multimodal.model_architecture import MultimodalModel

reproducible_comp()

2023-10-10 23:01:51.289758: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-10 23:01:51.289799: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-10 23:01:51.289821: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-10 23:01:51.296715: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-10 23:01:51.968502: W tensorflow/compiler/

In [3]:
file_path = "../../../data/processed_data/force_outer.pkl"
with open(file_path, 'rb') as picklefile:
    data = pickle.load(picklefile)

features_force_outer, category_force_outer, category_names, df_force_outer = get_final_df(data, is_acc_data=0, is_multilabel=1)

file_path = "../../../data/processed_data/acc_outer.pkl"
with open(file_path, 'rb') as picklefile:
    data = pickle.load(picklefile)

features_acc_outer, category_acc_outer, category_names, df_acc_outer = get_final_df(data, is_acc_data=1, is_multilabel=1)

In [4]:
labels_force = category_force_outer.copy()
features_force = features_force_outer.copy()
print("Force Features Shape: ", features_force.shape)
print("Force Labels Shape: ", labels_force.shape)

labels_acc = category_acc_outer.copy()
features_acc = features_acc_outer.copy()
print("Acceleration Features Shape: ", features_acc.shape)
print("Acceleration Labels Shape: ", labels_acc.shape)

Force Features Shape:  (1330, 1)
Force Labels Shape:  (1330, 4)
Acceleration Features Shape:  (1330, 3)
Acceleration Labels Shape:  (1330, 4)


#### For Force

In [5]:
max_length = 150
for i in range (len(features_force)):
    if(len(features_force[i][0]<max_length)):
        features_force[i][0]=np.pad(features_force[i][0], (0,max_length-len(features_force[i][0])), 'constant', constant_values=(0))
feat_force= np.zeros(len(features_force) *max_length).reshape(-1, max_length)
for i in range(len(features_force)):
    feat_force[i]= np.concatenate((features_force[i][0]), axis=None)
    

print("Updated Features Shape: ", feat_force.shape)

Updated Features Shape:  (1330, 150)


#### For Acceleration

In [6]:
max_length = 500

for i in range (len(features_acc)):
    if(len(features_acc[i][0]<max_length)):
        features_acc[i][0]=np.pad(features_acc[i][0], (0,max_length-len(features_acc[i][0])), 'constant', constant_values=(0))
        
    if(len(features_acc[i][1]<max_length)):
        features_acc[i][1]=np.pad(features_acc[i][1], (0,max_length-len(features_acc[i][1])), 'constant', constant_values=(0))
    
    if(len(features_acc[i][2]<max_length)):
        features_acc[i][2]=np.pad(features_acc[i][2], (0,max_length-len(features_acc[i][2])), 'constant', constant_values=(0))

feat_acc = np.zeros(len(features_acc)*3*max_length).reshape(-1, 3*max_length)
for i in range(len(features_acc)):
    feat_acc[i]= np.concatenate((features_acc[i][0],features_acc[i][1], features_acc[i][2]), axis=None)
    

print("Updated Features_acc Shape: ", feat_acc.shape)

Updated Features_acc Shape:  (1330, 1500)


In [7]:
X_train_force, X_test_force, y_train_force, y_test_force = train_test_split(feat_force, labels_force, test_size=0.1, 
                                                    random_state=2, shuffle=True, stratify=labels_force)

X_train_acc, X_test_acc, y_train_acc, y_test_acc = train_test_split(feat_acc, labels_acc, test_size=0.1, 
                                                    random_state=2, shuffle=True, stratify=labels_acc)                                                   

In [8]:
X_train_force = X_train_force.reshape(-1, 1, feat_force.shape[1])
X_test_force = X_test_force.reshape(-1, 1, feat_force.shape[1])

X_train_acc = X_train_acc.reshape(-1, 1, feat_acc.shape[1])
X_test_acc = X_test_acc.reshape(-1, 1, feat_acc.shape[1])

print(X_train_force.shape)
print(y_train_force.shape)
print(X_test_force.shape)
print(y_test_acc.shape)

print(X_train_acc.shape)
print(y_train_acc.shape)
print(X_test_acc.shape)
print(y_test_acc.shape)

(1197, 1, 150)
(1197, 4)
(133, 1, 150)
(133, 4)
(1197, 1, 1500)
(1197, 4)
(133, 1, 1500)
(133, 4)


In [9]:
from torch.utils.data import DataLoader, TensorDataset

input_channels = 1
num_classes = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your multimodal data and labels for both modalities
x_train_force = torch.tensor(X_train_force, dtype=torch.float32)
y_train_force = torch.tensor(y_train_force, dtype=torch.float32)

x_train_acc = torch.tensor(X_train_acc, dtype=torch.float32)
y_train_acc = torch.tensor(y_train_acc, dtype=torch.float32)

# Define the objective function for hyperparameter optimization
def multimodal_objective(trial: Trial):
    try:
        # Define hyperparameters to search for
        lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)  # Suggest a learning rate in log scale
        dropout_prob = trial.suggest_float("dropout_prob", 0.0, 0.5)
        force_n_layers = trial.suggest_int("force_n_layers", 1, 7)
        force_kernel_size = trial.suggest_int("force_kernel_size", 3, 9)
        force_stride = trial.suggest_int("force_stride", 1, 7)
        force_padding = trial.suggest_int("force_padding", 1, 10)
        force_hidden_channels = trial.suggest_int("force_hidden_channels", 16, 256)
        force_pooling_type = trial.suggest_categorical("force_pooling_type", ["max", "avg"])
        force_use_batch_norm = trial.suggest_categorical("force_use_batch_norm", [True, False])

        acc_n_layers = trial.suggest_int("acc_n_layers", 1, 7)
        acc_kernel_size = trial.suggest_int("acc_kernel_size", 3, 9)
        acc_stride = trial.suggest_int("acc_stride", 1, 7)
        acc_padding = trial.suggest_int("acc_padding", 1, 10)
        acc_hidden_channels = trial.suggest_int("acc_hidden_channels", 16, 256)
        acc_pooling_type = trial.suggest_categorical("acc_pooling_type", ["max", "avg"])
        acc_use_batch_norm = trial.suggest_categorical("acc_use_batch_norm", [True, False])


        # Split the data into training and validation sets
        x_train_split_force, x_valid_split_force, y_train_split_force, y_valid_split_force = train_test_split(
            x_train_force, y_train_force, test_size=0.2, random_state=42
        )
        x_train_split_acc, x_valid_split_acc, y_train_split_acc, y_valid_split_acc = train_test_split(
            x_train_acc, y_train_acc, test_size=0.2, random_state=42
        )

        # Create datasets and data loaders for both modalities
        train_dataset_force = TensorDataset(x_train_split_force, y_train_split_force)
        valid_dataset_force = TensorDataset(x_valid_split_force, y_valid_split_force)

        train_dataset_acc = TensorDataset(x_train_split_acc, y_train_split_acc)
        valid_dataset_acc = TensorDataset(x_valid_split_acc, y_valid_split_acc)

        train_loader_force = DataLoader(train_dataset_force, batch_size=64, shuffle=True)
        valid_loader_force = DataLoader(valid_dataset_force, batch_size=64, shuffle=False)

        train_loader_acc = DataLoader(train_dataset_acc, batch_size=64, shuffle=True)
        valid_loader_acc = DataLoader(valid_dataset_acc, batch_size=64, shuffle=False)

        # Create the multimodal model with the suggested hyperparameters
        model = MultimodalModel(input_channels, num_classes, 
                 force_n_layers, force_kernel_size, force_stride, force_padding, force_hidden_channels,
                 force_pooling_type, force_use_batch_norm,
                 acc_n_layers, acc_kernel_size, acc_stride, acc_padding, acc_hidden_channels,
                 acc_pooling_type, acc_use_batch_norm, dropout_prob).to(device)

        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        num_epochs = 50

        model.train()
        for epoch in range(num_epochs):
            # Training loop
            running_loss = 0.0
            for (inputs_acc, labels), (inputs_force,lab) in zip(train_loader_acc, train_loader_force):
                inputs_acc, labels, inputs_force = inputs_acc.to(device), labels.to(device), inputs_force.to(device)

                optimizer.zero_grad()

                outputs = model(inputs_force, inputs_acc)
                loss = criterion(outputs, labels.float())
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            train_loss = running_loss / len(train_loader_acc)

            # Validation loss calculation (similar to training loop)
            model.eval()
            valid_loss = 0.0
            with torch.no_grad():
                for (inputs_acc_valid, labels_valid), (inputs_force_valid,lab) in zip(valid_loader_acc, valid_loader_force):
                    inputs_acc_valid, labels_valid, inputs_force_valid = inputs_acc_valid.to(device), labels_valid.to(device), inputs_force_valid.to(device)
                    outputs_valid = model(inputs_force_valid, inputs_acc_valid)
                    loss_valid = criterion(outputs_valid, labels_valid.float())
                    valid_loss += loss_valid.item()

            valid_loss = valid_loss / len(valid_loader_acc)
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss}, Validation Loss: {valid_loss}")

        # Calculate and return the final validation loss
        return valid_loss
    
    except Exception as e:
        # Print the error for debugging purposes (optional)
        print(f"Trial failed with error: {e}")

        # Return a high validation loss to indicate failure
        return 1.0  # High validation loss


In [10]:
# Create an Optuna study for multimodal model hyperparameter optimization
study = optuna.create_study(
    study_name="Case#8: Multi Modal Outer",
    direction="minimize",  # Minimize final validation loss
    storage="sqlite:///../../database_sqllite//db.sqlite3",
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
)

# Define the number of trials (hyperparameter combinations) you want to search
num_trials = 150

# Perform hyperparameter optimization for the multimodal model
study.optimize(multimodal_objective, n_trials=num_trials, n_jobs=1)


[I 2023-10-10 23:03:32,003] A new study created in RDB with name: Case#8: Multi Modal Outer


Epoch 10/50, Training Loss: 0.2350373923778534, Validation Loss: 0.29982420057058334
Epoch 20/50, Training Loss: 0.15302783052126567, Validation Loss: 0.37698909640312195
Epoch 30/50, Training Loss: 0.10591483761866888, Validation Loss: 0.7381678596138954
Epoch 40/50, Training Loss: 0.04545615961154302, Validation Loss: 0.8121242821216583


[I 2023-10-10 23:07:03,680] Trial 0 finished with value: 1.144541174173355 and parameters: {'lr': 0.004635776945813943, 'dropout_prob': 0.4326538137174691, 'force_n_layers': 1, 'force_kernel_size': 9, 'force_stride': 6, 'force_padding': 8, 'force_hidden_channels': 189, 'force_pooling_type': 'max', 'force_use_batch_norm': False, 'acc_n_layers': 3, 'acc_kernel_size': 8, 'acc_stride': 1, 'acc_padding': 6, 'acc_hidden_channels': 213, 'acc_pooling_type': 'max', 'acc_use_batch_norm': False}. Best is trial 0 with value: 1.144541174173355.


Epoch 50/50, Training Loss: 0.013726228817055623, Validation Loss: 1.144541174173355


[I 2023-10-10 23:07:05,098] Trial 1 finished with value: 1.0 and parameters: {'lr': 0.005659013824908527, 'dropout_prob': 0.16041616887491572, 'force_n_layers': 5, 'force_kernel_size': 5, 'force_stride': 5, 'force_padding': 4, 'force_hidden_channels': 134, 'force_pooling_type': 'max', 'force_use_batch_norm': True, 'acc_n_layers': 6, 'acc_kernel_size': 6, 'acc_stride': 2, 'acc_padding': 7, 'acc_hidden_channels': 248, 'acc_pooling_type': 'max', 'acc_use_batch_norm': True}. Best is trial 1 with value: 1.0.


...

Epoch 50/50, Training Loss: 0.23243749837080638, Validation Loss: 0.2684036083519459
Epoch 10/50, Training Loss: 0.4075644771258036, Validation Loss: 0.38421402871608734
Epoch 20/50, Training Loss: 0.2965632647275925, Validation Loss: 0.30989692360162735
Epoch 30/50, Training Loss: 0.2753011226654053, Validation Loss: 0.268690075725317
Epoch 40/50, Training Loss: 0.2626774062712987, Validation Loss: 0.28125982731580734


[I 2023-10-11 00:18:26,826] Trial 148 finished with value: 0.26708516478538513 and parameters: {'lr': 0.0009404693280251217, 'dropout_prob': 0.41162972687976157, 'force_n_layers': 2, 'force_kernel_size': 3, 'force_stride': 1, 'force_padding': 3, 'force_hidden_channels': 32, 'force_pooling_type': 'max', 'force_use_batch_norm': True, 'acc_n_layers': 4, 'acc_kernel_size': 6, 'acc_stride': 2, 'acc_padding': 5, 'acc_hidden_channels': 71, 'acc_pooling_type': 'avg', 'acc_use_batch_norm': False}. Best is trial 78 with value: 0.25265607982873917.


Epoch 50/50, Training Loss: 0.2490180601676305, Validation Loss: 0.26708516478538513
Epoch 10/50, Training Loss: 0.3359015643596649, Validation Loss: 0.3126540184020996
Epoch 20/50, Training Loss: 0.27903297543525696, Validation Loss: 0.2708536684513092
Epoch 30/50, Training Loss: 0.26245160003503165, Validation Loss: 0.27505113184452057
Epoch 40/50, Training Loss: 0.2406601945559184, Validation Loss: 0.2771715335547924


[I 2023-10-11 00:18:59,380] Trial 149 finished with value: 0.2803432196378708 and parameters: {'lr': 0.0009307312799590769, 'dropout_prob': 0.4216175652357177, 'force_n_layers': 6, 'force_kernel_size': 3, 'force_stride': 1, 'force_padding': 3, 'force_hidden_channels': 32, 'force_pooling_type': 'max', 'force_use_batch_norm': False, 'acc_n_layers': 4, 'acc_kernel_size': 8, 'acc_stride': 2, 'acc_padding': 5, 'acc_hidden_channels': 84, 'acc_pooling_type': 'avg', 'acc_use_batch_norm': False}. Best is trial 78 with value: 0.25265607982873917.


Epoch 50/50, Training Loss: 0.21446880797545115, Validation Loss: 0.2803432196378708


In [11]:
# Print the best hyperparameters and their associated final validation loss
print("Best hyperparameters:")
print(study.best_params)
print("Best final validation loss:")
print(study.best_value)

Best hyperparameters:
{'lr': 0.0005452723485761036, 'dropout_prob': 0.42333881966763554, 'force_n_layers': 2, 'force_kernel_size': 3, 'force_stride': 5, 'force_padding': 4, 'force_hidden_channels': 65, 'force_pooling_type': 'max', 'force_use_batch_norm': True, 'acc_n_layers': 4, 'acc_kernel_size': 6, 'acc_stride': 2, 'acc_padding': 5, 'acc_hidden_channels': 33, 'acc_pooling_type': 'max', 'acc_use_batch_norm': False}
Best final validation loss:
0.25265607982873917


In [12]:
# Specify the path where you want to save the JSON file
config_file_path = "../best_configs/multimodal_outer_best_configs.json"

# Save the best hyperparameters and validation loss to a JSON file
best_configs = {
    "best_params": study.best_params,
    "best_value": study.best_value,
}

with open(config_file_path, "w") as config_file:
    json.dump(best_configs, config_file)

In [17]:

# Load the saved configuration
with open(config_file_path, "r") as config_file:
    saved_configs = json.load(config_file)

# Compare the new best value with the saved one
if study.best_value < saved_configs["best_value"]:
    print("New best value found. Replacing saved configuration.")
    saved_configs["best_params"] = study.best_params
    saved_configs["best_value"] = study.best_value

    # Update the saved configuration file
    with open(config_file_path, "w") as config_file:
        json.dump(saved_configs, config_file)

In [16]:
# Save the best hyperparameters and validation loss to a JSON file
best_configs = {
    "best_params": study.best_params,
    "best_value": study.best_value,
}

with open(config_file_path, "w") as config_file:
    json.dump(best_configs, config_file)

# Thank You